I've used this notebook to derive the BP algorithm on top of the PGMPY library, whose BP isn't a real BP algorithm

In [2]:
import src.models.builders as mb
import src.inference.helpers as ih
import numpy as np

from functools import reduce

In [3]:
# Create factor graph
height = 170
age = 30
sex = "Male"
model, inf_alg, HFEV1, ecFEV1, AR, HO2Sat, O2SatFFA, IA, UO2Sat, O2Sat = (
    mb.o2sat_fev1_point_in_time_model_2(height, age, sex)
)

buidling custom model (work in progress)


# BP message computation

Add this to the FactorGraph library

def point_mass_message(self, var, obs):
    """
    Returns the point mass message for the variable given the observed state.
    """
    card = self.get_cardinality(var)
    # Create an array with 1 at the index of the evidence and 0 elsewhere
    message = np.zeros(card)
    message[obs] = 1
    return message

In [3]:
# Functions moved to the custom BP algorithm
# def variable_node_message(incoming_messages):
#     """
#     The outgoing message is the element wise product of all incoming messages
#     """
#     if len(incoming_messages) == 1:
#         return incoming_messages[0]
#     outgoing_message = np.multiply(*incoming_messages)
#     # Normalise
#     return outgoing_message / np.sum(outgoing_message)


# def factor_node_message(incoming_messages, factor, target_var):
#     """
#     Returns the outgoing message for a factor node, which is the multiplication of the incoming messages with the factor function (CPT)

#     The variables' order in the incoming messages list must match the variable's order in the CPT's dimensions
#     """
#     cpt = factor.values
#     vars = factor.variables

#     assert (
#         len(incoming_messages) == cpt.ndim - 1
#     ), f"Error computing factor node message for {target_var}. The number of incoming messages must equal the card(CPT) - 1"

#     # Ensure that the target var is on the CPT's 0th axis
#     # Find idx of target var in the vars list
#     target_var_idx = vars.index(target_var)
#     if target_var_idx != 0:
#         # Move target var to the 0th axis
#         cpt = np.moveaxis(cpt, target_var_idx, 0)

#     # Invert incoming_messages, so that the first message corresponds to the last dimension of the CPT
#     incoming_messages = list(reversed(incoming_messages))

#     # Reduce the CPT with the inverted list of incoming messages
#     outgoing_message = reduce(
#         lambda cpt_reduced, m: np.matmul(cpt_reduced, m), incoming_messages, cpt
#     )

#     # Normalise
#     return outgoing_message / sum(outgoing_message)

In [4]:
var_to_infer = "Healthy FEV1 (L)"


ecFEV1_obs = 1
ecFEV1_idx = ecFEV1.get_bin_for_value(ecFEV1_obs)[1]

o2sat_obs = 100
o2sat_idx = O2Sat.get_bin_for_value(o2sat_obs)[1]

hfev1_factor = model.get_factors()[0]
evfev1_factor = model.get_factors()[1]
ar_factor = model.get_factors()[2]
ho2sat_factor = model.get_factors()[3]
o2satffa_factor = model.get_factors()[4]
ia_factor = model.get_factors()[5]
uo2sat_factor = model.get_factors()[6]
o2sat_factor = model.get_factors()[7]

# O2sat -> O2sat factor
o2sat_obs_m = model.point_mass_message(O2Sat.name, o2sat_idx)
o2sat_up = inf_alg.variable_node_message([o2sat_obs_m])

# Only one message comes to UO2Sat
to_uo2sat = inf_alg.factor_node_message([o2sat_up], o2sat_factor, UO2Sat.name)
# Only one message incoming UO2Sat, hence
uo2sat_up = inf_alg.variable_node_message([to_uo2sat])
assert (uo2sat_up == to_uo2sat).all()

# IA -> O2Sat factor
ia_right = inf_alg.variable_node_message([IA.cpt])

# uo2sat_factor -> o2satffa
to_o2satffa = inf_alg.factor_node_message(
    [uo2sat_up, ia_right], uo2sat_factor, O2SatFFA.name
)

# O2SatFFA to O2SatFFA factor
o2satffa_up = inf_alg.variable_node_message([to_o2satffa])
assert (o2satffa_up == to_o2satffa).all()

# HO2Sat to O2SatFFA factor
ho2sat_down = inf_alg.variable_node_message([HO2Sat.cpt])

# O2SatFFA factor to AR
to_ar = inf_alg.factor_node_message(
    [o2satffa_up, ho2sat_down], o2satffa_factor, AR.name
)

# AR -> ecfev1 factor
ar_left = inf_alg.variable_node_message([AR.cpt, to_ar])

# ecFEV1 -> ecfev1 factor
ecfev1_obs_m = model.point_mass_message(ecFEV1.name, ecFEV1_idx)
ecfev1_up = inf_alg.variable_node_message([ecfev1_obs_m])

# ecfev1 -> hfev1
to_hfev1 = inf_alg.factor_node_message(
    [ecfev1_up, ar_left], model.get_factors()[1], var_to_infer
)

# hfev1 posterior
hfev1_posterior = inf_alg.variable_node_message([to_hfev1, HFEV1.cpt])
hfev1_posterior.shape

(100,)

In [5]:
# Compare results against the bayes net model (factor graph model takes ages to compute)
model1, inf_alg1, _, _, _, _, _, _, _, _ = mb.o2sat_fev1_point_in_time_model(
    height, age, sex
)
res = ih.infer(inf_alg1, [HFEV1], [[ecFEV1, ecFEV1_obs], [O2Sat, o2sat_obs]])
res[HFEV1.name].values.shape

/Users/tristan.trebaol/Desktop/PhD/Code/pgmpy/pgmpy/factors/discrete/DiscreteFactor.py:796: RuntimeWarning: invalid value encountered in divide
  phi.values = phi.values / phi1.values


(100,)

In [6]:
((hfev1_posterior - res[HFEV1.name].values) < 1e-5).all()
# Plot the two posteriors with plotly go figure
import plotly.graph_objs as go

fig = go.Figure()
# Add bar plot for the factor graph model
fig.add_trace(go.Bar(x=HFEV1.bins, y=hfev1_posterior, name="Factor Graph"))
fig.add_trace(go.Bar(x=HFEV1.bins, y=res[HFEV1.name].values, name="Bayes Net"))
# Update layout
fig.update_layout(
    barmode="group", title="Comparison of HFEV1 posteriors", width=800, height=300
)
fig.show()